In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd, numpy as np
from tqdm.auto import tqdm
import os, sys, pickle, glob, gc, shutil
import math
from pathlib import Path


from collections import Counter
import itertools

from eventsSuggesterNew import ClicksSuggester, BuysSuggester, CartsSuggester


import gensim

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'
! mkdir {outputPath}

mkdir: cannot create directory ‘../output/newSplited/’: File exists


In [3]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

# Generate candidates

In [5]:
TOPN_clicks = 20
TOPN_b2b = 20
TOPN_cartsOrders = 20
TOPN_c2c = 20
TOPN_click2cart = 20
TOPN_click2order = 20

candidatesNum = 100
SET = 2
VER = 6
note_covisit = 'covisit_20_20_20'
note_candidate = 'suggester_addLast'

## Load testA data

In [6]:
def load_test(path):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(path)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})
    
if SET == 1:
    testA = load_test('../input/split_2-1_pqt/test_parquets/*')
elif SET == 2:
    testA = pd.read_parquet('../input/splited/test.parquet')

elif SET == 3:
    testA = load_test('../input/parquets/test_parquets/*')
testA

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


## Load covistation matrix

In [7]:
coVisitSaveFolder = outputPath + f'/coVisit/set{SET}/top20_20_20/'

In [8]:
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

In [9]:
top_clicks_covisit = {}
for k in range(0, 8):
    top_clicks_covisit.update( pqt_to_dict( pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_clicks_v{VER}_{k}.pqt') ) )

top_c2c_covisit = {}
for k in range(0, 8): 
    top_c2c_covisit.update( pqt_to_dict( pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_c2c}_click2click_v{VER}_{k}.pqt') ) )

In [9]:
top_buy2buy_covisit = {}
for k in range(0, 2):
    top_buy2buy_covisit.update( pqt_to_dict( pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_b2b}_buy2buy_v{VER}_{k}.pqt') ) )

top_cartsOrders_covisit = {}
for k in range(0, 8): 
    top_cartsOrders_covisit.update( pqt_to_dict( pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_cartsOrders}_cartsOrders_v{VER}_{k}.pqt') ) )

top_click2cart_covisit = {}
for k in range(0, 8): 
    top_click2cart_covisit.update( pqt_to_dict( pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2cart}_click2cart_v{VER}_{k}.pqt') ) )

top_click2order_covisit = {}
for k in range(0, 8): 
    top_click2order_covisit.update( pqt_to_dict( pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2order}_click2order_v{VER}_{k}.pqt') ) )

/tmp/ipykernel_7041/1906639434.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby('aid_x').aid_y.apply(list).to_dict()
/tmp/ipykernel_7041/1906639434.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby('aid_x').aid_y.apply(list).to_dict()
/tmp/ipykernel_7041/

## find most frequently item

In [11]:
top_clicks = testA.loc[testA['type']==0,'aid'].value_counts().index.values[:candidatesNum].astype(np.int32)
top_carts = testA.loc[testA['type'] == 1,'aid'].value_counts().index.values[:candidatesNum].astype(np.int32)
top_orders = testA.loc[testA['type'] == 2,'aid'].value_counts().index.values[:candidatesNum].astype(np.int32)

# print('Here are size of our 3 co-visitation matrices:')
# print( len( top_clicks_covisit ), len( top_buy2buy_covisit ), len( top_cartsOrders_covisit ) )

In [11]:
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format(f'../output/newSplited/savedModel/set_{SET}/otto_aid2vec_5d.bin', binary=True)

# suggest

In [12]:
clicksSuggester = ClicksSuggester(top_clicks, top_c2c_covisit, top_clicks_covisit, aid2vec_model)
cartsSuggester = CartsSuggester(top_carts, top_buy2buy_covisit, top_cartsOrders_covisit, top_click2cart_covisit, aid2vec_model)
ordersSuggester = BuysSuggester(top_orders, top_buy2buy_covisit, top_cartsOrders_covisit, top_click2order_covisit, aid2vec_model)

In [13]:
candidatesSavePath = outputPath + f'candidates/set{SET}_top_{candidatesNum}/{note_candidate}/'

try: 
    os.makedirs(candidatesSavePath) 
except OSError as error: 
    print(error)

In [14]:
def saveChunk(data, chunkSize, path, predType):
    n = len(data)
    i = 0
    while(i*chunkSize < n):
        sub = data[i*chunkSize:(i+1)*chunkSize]
        sub.to_parquet(path + f'{predType}_{i}.pqt')
        i += 1

### clicks

In [ ]:
tqdm.pandas()
pred_df_clicks = testA.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(lambda x: clicksSuggester.suggest(x, candidatesNum)).to_frame().reset_index()
# pred_df_clicks = testA.sort_values(["session", "ts"]).groupby(["session"]).parallel_apply(lambda x: clicksSuggester.suggest(x, TOPN_candidate)).to_frame().reset_index()
pred_df_clicks.columns = ['session', 'labels']
pred_df_clicks['session'] = pred_df_clicks['session'].astype('int32')
pred_df_clicks

In [ ]:
aids = pred_df_clicks.labels.explode().astype('int32').rename('aid')
clicksCandidate = pred_df_clicks[['session']].astype('int32')
clicksCandidate = clicksCandidate.merge(aids, left_index=True, right_index=True, how='left').reset_index(drop=True)
saveChunk(clicksCandidate, 650000 * 100, candidatesSavePath, 'clicks')

### carts

In [ ]:
tqdm.pandas()
pred_df_carts = testA.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(lambda x: cartsSuggester.suggest(x, candidatesNum)).to_frame().reset_index()
pred_df_carts.columns = ['session', 'labels']
pred_df_carts['session'] = pred_df_carts['session'].astype('int32')
pred_df_carts

A Jupyter Widget

,session,labels
0,12899779,"[59625, 469285, 689970, 159257, 731692, 397451..."
1,12899780,"[1142000, 736515, 973453, 582732, 889686, 1758..."
2,12899781,"[918667, 199008, 194067, 57315, 141736, 89022,..."
3,12899782,"[834354, 595994, 740494, 889671, 987399, 10076..."
4,12899783,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577,"[1141710, 86916, 768043, 1666114, 1681772, 935..."
1671799,14571578,"[519105, 830061, 290137, 1084758, 815460, 6913..."
1671800,14571579,"[739876, 702275, 1750859, 1286038, 1550479, 78..."
1671801,14571580,"[202353, 1314576, 1231403, 925638, 1295081, 38..."


In [ ]:
aids = pred_df_carts.labels.explode().astype('int32').rename('aid')
cartsCandidate = pred_df_carts[['session']].astype('int32')
cartsCandidate = cartsCandidate.merge(aids, left_index=True, right_index=True, how='left').reset_index(drop=True)
saveChunk(cartsCandidate, 650000 * 100, candidatesSavePath, 'carts')

### orders

In [ ]:
tqdm.pandas()
pred_df_orders = testA.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(lambda x: ordersSuggester.suggest(x, candidatesNum)).to_frame().reset_index()
# pred_df_buys = testA.sort_values(["session", "ts"]).groupby(["session"]).parallel_apply(lambda x: buysSuggester.suggest(x, TOPN_candidate)).to_frame().reset_index()
pred_df_orders.columns = ['session', 'labels']
pred_df_orders['session'] = pred_df_orders['session'].astype('int32')
pred_df_orders

A Jupyter Widget

,session,labels
0,12899779,"[59625, 689970, 159257, 731692, 397451, 125352..."
1,12899780,"[1142000, 736515, 973453, 582732, 889686, 4871..."
2,12899781,"[918667, 199008, 194067, 57315, 141736, 89022,..."
3,12899782,"[834354, 740494, 987399, 889671, 779477, 12740..."
4,12899783,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577,"[1141710, 1276792, 631085, 1004292, 842555, 11..."
1671799,14571578,"[519105, 815460, 977826, 822641, 1811714, 1321..."
1671800,14571579,"[739876, 702275, 1434015, 770418, 1750859, 513..."
1671801,14571580,"[202353, 1314576, 888228, 433425, 1231403, 925..."


In [ ]:
aids = pred_df_orders.labels.explode().astype('int32').rename('aid')
ordersCandidate = pred_df_orders[['session']].astype('int32')
ordersCandidate = ordersCandidate.merge(aids, left_index=True, right_index=True, how='left').reset_index(drop=True)
saveChunk(ordersCandidate, 650000 * 100, candidatesSavePath, 'orders')

# Check recall rate of candidates (for set == 2 or set ==1)

## Trainset

In [ ]:
pred_df_clicks['type'] = 'clicks'
pred_df_clicks

,session,labels,type
0,11098528,"[11830, 588923, 876129, 1732105, 307904, 20535...",clicks
1,11098529,"[1105029, 297916, 459126, 217742, 1717995, 138...",clicks
2,11098530,"[409236, 264500, 963957, 583026, 210880, 75233...",clicks
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",clicks
4,11098532,"[876469, 7651, 669555, 77906, 1159379, 1704066...",clicks
...,...,...,...
1801246,12899774,"[33035, 771913, 31490, 1399483, 1053112, 86143...",clicks
1801247,12899775,"[1743151, 155954, 1550204, 1760714, 1022572, 3...",clicks
1801248,12899776,"[548599, 1150130, 793373, 1401030, 1440959, 51...",clicks
1801249,12899777,"[384045, 1308634, 1450928, 1281056, 1688215, 1...",clicks


In [ ]:
pred_df_carts['type'] = 'carts'
pred_df_carts

,session,labels,type
0,11098528,"[11830, 588923, 876129, 1732105, 307904, 88450...",carts
1,11098529,"[1105029, 982457, 778200, 441348, 295362, 9241...",carts
2,11098530,"[409236, 264500, 963957, 583026, 364155, 25415...",carts
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",carts
4,11098532,"[876469, 7651, 669555, 77906, 1159379, 1704066...",carts
...,...,...,...
1801246,12899774,"[33035, 819288, 1089446, 1167677, 193794, 7719...",carts
1801247,12899775,"[1743151, 155954, 1760714, 1022572, 310546, 17...",carts
1801248,12899776,"[548599, 1150130, 1440959, 1607333, 1330306, 1...",carts
1801249,12899777,"[384045, 1308634, 1515273, 1281056, 1688215, 3...",carts


In [ ]:
pred_df_orders['type'] = 'orders'
pred_df_orders

,session,labels,type
0,11098528,"[11830, 588923, 876129, 1732105, 307904, 88450...",orders
1,11098529,"[1105029, 628708, 792887, 1436233, 982457, 890...",orders
2,11098530,"[409236, 264500, 1268065, 963957, 583026, 2108...",orders
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",orders
4,11098532,"[876469, 7651, 669555, 77906, 1159379, 1704066...",orders
...,...,...,...
1801246,12899774,"[33035, 771913, 31490, 1399483, 1053112, 86143...",orders
1801247,12899775,"[1743151, 155954, 1760714, 1022572, 310546, 17...",orders
1801248,12899776,"[548599, 177902, 1150130, 1533919, 861477, 140...",orders
1801249,12899777,"[384045, 1308634, 1821821, 1515273, 1450928, 1...",orders


In [ ]:
pred_df = pd.concat([pred_df_clicks, pred_df_carts, pred_df_orders]).reset_index(drop=True)
pred_df

,session,labels,type
0,11098528,"[11830, 588923, 876129, 1732105, 307904, 20535...",clicks
1,11098529,"[1105029, 297916, 459126, 217742, 1717995, 138...",clicks
2,11098530,"[409236, 264500, 963957, 583026, 210880, 75233...",clicks
3,11098531,"[396199, 1271998, 452188, 1728212, 1365569, 62...",clicks
4,11098532,"[876469, 7651, 669555, 77906, 1159379, 1704066...",clicks
...,...,...,...
5403748,12899774,"[33035, 771913, 31490, 1399483, 1053112, 86143...",orders
5403749,12899775,"[1743151, 155954, 1760714, 1022572, 310546, 17...",orders
5403750,12899776,"[548599, 177902, 1150130, 1533919, 861477, 140...",orders
5403751,12899777,"[384045, 1308634, 1821821, 1515273, 1450928, 1...",orders


In [ ]:
if SET == 1:
    test_label_path = '../input/split_2-1_pqt/test_labels.parquet'
elif SET == 2:
    test_label_path = '../input/splited/test_labels.parquet'
test_labels = pd.read_parquet(test_label_path)

In [ ]:
test_labels = test_labels.merge(pred_df, how='left', on=['session', 'type'])
test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels[:20]))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
test_labels

,session,type,ground_truth,labels,hits,gt_count
0,11098528,clicks,[1679529],"[11830, 588923, 876129, 1732105, 307904, 20535...",0,1
1,11098528,carts,[1199737],"[11830, 588923, 876129, 1732105, 307904, 88450...",0,1
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...","[11830, 588923, 876129, 1732105, 307904, 88450...",1,11
3,11098529,clicks,[1105029],"[1105029, 297916, 459126, 217742, 1717995, 138...",1,1
4,11098530,orders,[409236],"[409236, 264500, 1268065, 963957, 583026, 2108...",1,1
...,...,...,...,...,...,...
2212687,12899774,clicks,[1399483],"[33035, 771913, 31490, 1399483, 1053112, 86143...",1,1
2212688,12899775,clicks,[1760714],"[1743151, 155954, 1550204, 1760714, 1022572, 3...",1,1
2212689,12899776,clicks,[1737908],"[548599, 1150130, 793373, 1401030, 1440959, 51...",0,1
2212690,12899777,clicks,[384045],"[384045, 1308634, 1450928, 1281056, 1688215, 1...",1,1


In [ ]:
print('Set:', SET, 'Top:', candidatesNum)
recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum()
print(recall_per_type)
score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
print(score)

Set: 2 Top: 100
type
carts     0.419272
clicks    0.536148
orders    0.652270
dtype: float64
0.5707582388019186


## Validate on small dataset

In [34]:
123

[autoreload of eventsSuggesterNew failed: Traceback (most recent call last):
  File "/opt/conda/envs/rapids/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/envs/rapids/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/conda/envs/rapids/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/opt/conda/envs/rapids/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    update_instances(old, new)
  File "/opt/conda/envs/rapids/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    refs = gc.get_referrers(old)
KeyboardInterrupt
]


123

In [11]:
SET = 2
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format(f'../output/newSplited/savedModel/set_{SET}/otto_aid2vec_5d.bin', binary=True)

In [322]:
testA = pd.read_parquet('../input/splited/test.parquet')
top_clicks = testA.loc[testA['type']==0,'aid'].value_counts().index.values[:100].astype(np.int32)
top_carts = testA.loc[testA['type'] == 1,'aid'].value_counts().index.values[:100].astype(np.int32)
top_orders = testA.loc[testA['type'] == 2,'aid'].value_counts().index.values[:100].astype(np.int32)

In [341]:
import random
from eventsSuggesterNew import ClicksSuggester, BuysSuggester, CartsSuggester

testA = pd.read_parquet('../input/splited/test.parquet')
random.seed(10)
uniqueUser = list(testA.session.unique())
keepUsers = random.sample(uniqueUser, int(len(uniqueUser)/10))
testA = testA[testA.session.isin(keepUsers)].reset_index(drop=True)

predictType = 'orders'
# suggester = CartsSuggester(top_carts, top_buy2buy_covisit, top_cartsOrders_covisit, top_click2cart_covisit, aid2vec_model)
suggester = BuysSuggester(top_orders, top_buy2buy_covisit, top_cartsOrders_covisit, top_click2order_covisit, aid2vec_model)
suggester.a
# suggester = ClicksSuggester(top_clicks, top_c2c_covisit, top_clicks_covisit, aid2vec_model)

3

In [342]:
tqdm.pandas()
pred_df = testA.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(lambda x: suggester.suggest(x, candidatesNum)).to_frame().reset_index()
pred_df.columns = ['session', 'labels']
pred_df['session'] = pred_df['session'].astype('int32')
pred_df

A Jupyter Widget

,session,labels
0,11098529,"[1105029, 1135201, 295362, 441348, 1049489, 51..."
1,11098540,"[1545215, 1277598, 896787, 789082, 1472725, 10..."
2,11098553,"[314297, 1061776, 399992, 1783610, 1125095, 57..."
3,11098557,"[1234327, 719075, 1471333, 1732776, 53060, 527..."
4,11098559,"[907069, 84703, 1003267, 607328, 1381738, 5999..."
...,...,...
180120,12899695,"[206768, 1712543, 1005385, 1572478, 413094, 18..."
180121,12899714,"[559593, 138753, 683268, 472892, 1839548, 5541..."
180122,12899719,"[1217083, 250637, 132109, 226025, 508883, 1156..."
180123,12899763,"[1539032, 750901, 95488, 873385, 1854775, 3695..."


In [343]:
pred_df['type'] = predictType
test_labels = pd.read_parquet('../input/splited/test_labels.parquet')
type_labels = test_labels[test_labels['type'] == predictType]
type_labels = type_labels.merge(pred_df, how='left', on=['session', 'type']).dropna()
type_labels['hits'] = type_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
type_labels['gt_count'] = type_labels.ground_truth.str.len().clip(0,20)
recall_per_type = type_labels['hits'].sum() / type_labels['gt_count'].sum()
recall_per_type

0.6915200924380537

In [344]:
pred_df['type'] = predictType
test_labels = pd.read_parquet('../input/splited/test_labels.parquet')
type_labels = test_labels[test_labels['type'] == predictType]
type_labels = type_labels.merge(pred_df, how='left', on=['session', 'type']).dropna()
type_labels['hits'] = type_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels[:20]))), axis=1)
type_labels['gt_count'] = type_labels.ground_truth.str.len().clip(0,20)
recall_per_type20 = type_labels['hits'].sum() / type_labels['gt_count'].sum()
recall_per_type20

0.6517203748876621

0.6453853787628184

/10

0.6890807549107716

0.6516882783412504

## old

In [ ]:
0.6877647965078958
0.6505328026704327

0.47257675060365645

## now

In [ ]:
0.6885993067145975
0.6516882783412504

0.47440496723007936

new

In [ ]:
0.602599650419326
0.5365550931171323

In [ ]:
0.5057669362322683
0.4402757523531751

In [ ]:
0.6920978302734626
0.651399409423546

yesterday

0.5381151111085807

0.4205588133839255

0.6516882783412504

In [ ]:
0.4205588133839255*0.3 + 0.5381151111085807*0.1 + 0.6516882783412504*0.6

0.570992122130786

## Submit only covisitation candidates (for SET == 3)

In [ ]:
pred_df_clicks.session = pred_df_clicks.session.apply(lambda x: str(x) + '_' + 'clicks')
pred_df_clicks.labels = pred_df_clicks.labels.apply(lambda x: x[:20])

pred_df_clicks

,session,labels
0,12899779_clicks,"[59625, 1253524, 737445, 438191, 731692, 17907..."
1,12899780_clicks,"[1142000, 736515, 973453, 582732, 1502122, 889..."
2,12899781_clicks,"[918667, 199008, 194067, 57315, 141736, 146057..."
3,12899782_clicks,"[1007613, 595994, 1033148, 834354, 479970, 169..."
4,12899783_clicks,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577_clicks,"[1141710, 1276792, 1004292, 1666114, 367734, 8..."
1671799,14571578_clicks,"[519105, 977826, 1811714, 822641, 815460, 1536..."
1671800,14571579_clicks,"[739876, 1209992, 1750859, 1550479, 51363, 785..."
1671801,14571580_clicks,"[202353, 1314576, 433425, 871658, 1231403, 925..."


In [ ]:
pred_df_carts.session = pred_df_carts.session.apply(lambda x: str(x) + '_' + 'carts')
pred_df_carts.labels = pred_df_carts.labels.apply(lambda x: x[:20])

pred_df_carts

,session,labels
0,12899779_carts,"[59625, 397451, 469285, 689970, 1493965, 48525..."
1,12899780_carts,"[1142000, 736515, 973453, 582732, 487136, 1263..."
2,12899781_carts,"[918667, 199008, 194067, 57315, 141736, 759436..."
3,12899782_carts,"[1007613, 595994, 1033148, 834354, 479970, 169..."
4,12899783_carts,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577_carts,"[1141710, 86916, 1276792, 1006198, 631085, 768..."
1671799,14571578_carts,"[519105, 977826, 664851, 1498443, 1736598, 652..."
1671800,14571579_carts,"[739876, 1209992, 1550479, 785544, 1750859, 87..."
1671801,14571580_carts,"[202353, 387358, 433425, 528665, 575020, 84809..."


In [ ]:
pred_df_orders.session = pred_df_orders.session.apply(lambda x: str(x) + '_' + 'orders')
pred_df_orders.labels = pred_df_orders.labels.apply(lambda x: x[:20])

pred_df_orders

,session,labels
0,12899779_orders,"[59625, 485256, 986164, 1460571, 33343, 554660..."
1,12899780_orders,"[1142000, 736515, 973453, 582732, 487136, 1758..."
2,12899781_orders,"[918667, 199008, 194067, 57315, 141736, 146057..."
3,12899782_orders,"[1007613, 595994, 1033148, 834354, 479970, 169..."
4,12899783_orders,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577_orders,"[1141710, 1006198, 1432586, 1523164, 485256, 9..."
1671799,14571578_orders,"[519105, 664851, 1498443, 1736598, 485256, 986..."
1671800,14571579_orders,"[739876, 785544, 876416, 1750859, 12334, 12479..."
1671801,14571580_orders,"[202353, 336919, 14540, 32322, 79147, 105517, ..."


In [ ]:
pred_df = pd.concat([pred_df_clicks, pred_df_carts, pred_df_orders])
pred_df.columns = ["session_type", "labels"]
pred_df["labels"] = pred_df.labels.apply(lambda x: " ".join(map(str,x)))
pred_df

,session_type,labels
0,12899779_clicks,59625 1253524 737445 438191 731692 1790770 942...
1,12899780_clicks,1142000 736515 973453 582732 1502122 889686 48...
2,12899781_clicks,918667 199008 194067 57315 141736 1460571 7594...
3,12899782_clicks,1007613 595994 1033148 834354 479970 1696036 8...
4,12899783_clicks,1817895 607638 1754419 1216820 1729553 300127 ...
...,...,...
1671798,14571577_orders,1141710 1006198 1432586 1523164 485256 986164 ...
1671799,14571578_orders,519105 664851 1498443 1736598 485256 986164 14...
1671800,14571579_orders,739876 785544 876416 1750859 12334 124793 1286...
1671801,14571580_orders,202353 336919 14540 32322 79147 105517 218014 ...


In [ ]:
pred_df.to_csv(f'../submissions/covisitCandidates_{note_candidate}.csv', index=False)

## Top50 to top20

In [ ]:
subs = []
clicks_pred_df = pd.read_parquet('../output/newSplited/candidates_set3_top50_covisit_15_20_15/clicksCandidate.pqt')
carts_pred_df = pd.read_parquet('../output/newSplited/candidates_set3_top50_covisit_15_20_15/buysCandidate.pqt')
orders_pred_df = pd.read_parquet('../output/newSplited/candidates_set3_top50_covisit_15_20_15/buysCandidate.pqt')


In [ ]:
clicks_pred_df.session = clicks_pred_df.session.apply(lambda x: str(x) + '_' + 'clicks')
clicks_pred_df.labels = clicks_pred_df.labels.apply(lambda x: x[:20])

In [ ]:
carts_pred_df.session = carts_pred_df.session.apply(lambda x: str(x) + '_' + 'carts')
carts_pred_df.labels = carts_pred_df.labels.apply(lambda x: x[:20])

In [ ]:
orders_pred_df.session = orders_pred_df.session.apply(lambda x: str(x) + '_' + 'orders')
orders_pred_df.labels = orders_pred_df.labels.apply(lambda x: x[:20])

In [ ]:
pred_df = pd.concat([clicks_pred_df, carts_pred_df, orders_pred_df])
pred_df.columns = ["session_type", "labels"]
pred_df["labels"] = pred_df.labels.apply(lambda x: " ".join(map(str,x)))
pred_df

,session_type,labels
0,12899779_clicks,59625 1253524 737445 438191 731692 1790770 942...
1,12899780_clicks,1142000 736515 973453 582732 1502122 889686 48...
2,12899781_clicks,918667 199008 194067 57315 141736 1460571 7594...
3,12899782_clicks,1007613 595994 1033148 834354 479970 1696036 8...
4,12899783_clicks,1817895 607638 1754419 1216820 1729553 300127 ...
...,...,...
1671798,14571577_orders,1141710 1276792 1666114 86916 631085 1004292 3...
1671799,14571578_orders,519105 977826 1811714 822641 1671592 815460 15...
1671800,14571579_orders,739876 1209992 1750859 1550479 785544 51363 21...
1671801,14571580_orders,202353 433425 1314576 1231403 871658 925638 89...


In [ ]:
pred_df.to_csv(f'../submissions/covisitCandidates_50to20.csv', index=False)